In [1]:
from gensim import models
import numpy as np

In [2]:
first_model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin\GoogleNews-vectors-negative300.bin', binary=True)

In [3]:
first_model.most_similar('man')

[('woman', 0.7664012908935547),
 ('boy', 0.6824871301651001),
 ('teenager', 0.6586930155754089),
 ('teenage_girl', 0.6147903203964233),
 ('girl', 0.5921714305877686),
 ('suspected_purse_snatcher', 0.571636438369751),
 ('robber', 0.5585119128227234),
 ('Robbery_suspect', 0.5584409832954407),
 ('teen_ager', 0.5549196600914001),
 ('men', 0.5489763021469116)]

In [4]:
first_model.sort_by_descending_frequency()
first_model.unit_normalize_all()

In [5]:
first_model.index_to_key[:10]

['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said']

In [6]:
def verifier_chaine(chaine) :
    if chaine.islower() and chaine.isalpha() and len(chaine)<20 :
        return True
    return False

In [7]:
words_sorted = first_model.index_to_key[:50000]
words_sorted = [word for word in words_sorted if verifier_chaine(word)]
words_sorted[:10], len(words_sorted)

(['in', 'for', 'that', 'is', 'on', 'with', 'said', 'was', 'the', 'at'], 24065)

In [8]:
vector_size = first_model.vector_size
model = models.keyedvectors.KeyedVectors(vector_size, count=0)
for word in words_sorted :
    model[word] = first_model[word]

In [9]:
model.most_similar('man')

[('woman', 0.7664012908935547),
 ('boy', 0.6824870109558105),
 ('teenager', 0.6586930155754089),
 ('girl', 0.5921714305877686),
 ('robber', 0.5585119128227234),
 ('men', 0.5489763021469116),
 ('guy', 0.5420035719871521),
 ('person', 0.5342026352882385),
 ('gentleman', 0.5337990522384644),
 ('motorcyclist', 0.5336882472038269)]

**Rappels de concepts statistiques** : 

1. La p-valeur 

La p-valeur, ou valeur de probabilité, est un concept important en statistique qui est utilisé pour évaluer la validité d'une hypothèse nulle dans le cadre d'un test statistique. Elle est souvent associée aux tests d'hypothèses et joue un rôle crucial dans la prise de décisions statistiques.

Voici comment la p-valeur fonctionne :

1. **Formulation des hypothèses :** Lorsqu'on effectue un test statistique, on formule généralement deux hypothèses. L'hypothèse nulle (H0) stipule qu'il n'y a pas d'effet ou de différence, tandis que l'hypothèse alternative (H1) suggère qu'il y a un effet ou une différence.

2. **Collecte des données et calcul de la statistique de test :** On collecte des données et on calcule une statistique de test à partir de ces données. Cette statistique de test dépend du type de test statistique que l'on utilise (par exemple, le test t, le test de chi-carré, le test F, etc.).

3. **Calcul de la p-valeur :** La p-valeur est la probabilité d'observer une statistique de test aussi extrême, ou plus extrême, que celle obtenue, sous l'hypothèse nulle. En d'autres termes, c'est la probabilité que les résultats observés soient dus au hasard, sachant que l'hypothèse nulle est vraie.

4. **Comparaison avec le seuil de signification :** On compare ensuite la p-valeur à un seuil de signification prédéfini, généralement noté alpha (α). Si la p-valeur est inférieure à alpha, on rejette l'hypothèse nulle au profit de l'hypothèse alternative. Si la p-valeur est supérieure à alpha, on ne rejette pas l'hypothèse nulle.

En résumé, une petite p-valeur suggère que les résultats observés sont peu probables sous l'hypothèse nulle, ce qui conduit à la remise en question de cette hypothèse. En revanche, une grande p-valeur indique que les résultats sont compatibles avec l'hypothèse nulle, et il n'y a pas suffisamment de preuves pour la rejeter.

Il est essentiel de noter que la p-valeur ne fournit pas la probabilité que l'hypothèse nulle soit vraie ou fausse, mais plutôt la probabilité d'observer les données actuelles ou quelque chose de plus extrême si l'hypothèse nulle est vraie.

2. Taille d'effet 

vidéo sur la cohen's d method : https://www.youtube.com/watch?v=IetVSlrndpI

**Etape 1** : coder le test de WEAT

In [10]:
def word_to_vector(list_word, model) :
    #list_word doit être une liste de string
    arr_vector = np.zeros((len(list_word), model.vector_size))
    for i, word in enumerate(list_word) :
        arr_vector[i] = model[word]
    return arr_vector

In [11]:
#contrairement à l'article, j'ai pas equationS dans mon vocabulaire
X1 = ['math', 'algebra', 'geometry', 'calculus', 'equation', 'computation', 'numbers', 'addition']
X1_vec = word_to_vector(X1, model)

In [13]:
B1 = ['female', 'woman', 'girl', 'sister', 'she', 'her', 'hers', 'daughter']
B1_vec = word_to_vector(B1, model)
for i in range(np.shape(X1_vec)[0]) :
    sim_x_B1 = model.cosine_similarities(X1_vec[i], B1_vec)
print(sim_x_B1.shape)


(8,)


In [14]:
moyenne = np.mean(sim_x_B1)
print(moyenne)

-0.013237026812653103


In [15]:
def s_w_A_B(w, A, B, model) :
    #w est déjà un vecteur de mot
    sim_w_A = model.cosine_similarities(w, A)
    mean_w_A = np.mean(sim_w_A)
    sim_w_B = model.cosine_similarities(w, B)
    mean_w_B = np.mean(sim_w_B)
    return mean_w_A - mean_w_B

In [16]:
A1 = ['male', 'man', 'boy', 'brother', 'he', 'him', 'his', 'son']
A1_vec = word_to_vector(A1, model)
s_w_A_B(X1_vec[0], B1_vec, A1_vec, model)

0.0432115584222808

In [17]:
def test_weat(X,Y,A,B, model): #v2 permet de déjà débuter le calcul de l'effectsize qui devient très redondant sinon
    # X,Y sont des listes de mots_test de même taille
    #ex : X = math words(calculus, equations, etc.) et Y = arts words (poetry, art, etc.)
    # A,B are lists of attribute words
    #ex : A = female attributes (women, sister, girl, etc.) et B = male attributes (man, boy, etc.)
    X = word_to_vector(X, model)
    Y = word_to_vector(Y, model)
    A = word_to_vector(A, model)
    B = word_to_vector(B, model)
    #j'utilise la cosinus_similarité, donc les vecteurs n'ont pas besoin d'être normalisés
    sum_x = 0
    sum_y = 0
    mean_x = 0 #les calculs de moyenne servent par la suite pour le calcul de l'effectsize
    mean_y = 0
    mean_glob = 0 
    for i in range(np.shape(X)[0]) :
        xi_AB = s_w_A_B(X[i], A, B, model)
        sum_x += xi_AB
        mean_x += xi_AB
        mean_glob += xi_AB
    for i in range(np.shape(Y)[0]) :    
        yi_AB = s_w_A_B(Y[i], A, B, model)
        sum_y += yi_AB
        mean_y += yi_AB
        mean_glob += yi_AB
    mean_x = mean_x/np.shape(X)[0]
    mean_y = mean_y/np.shape(Y)[0]
    mean_glob = mean_glob/(np.shape(X)[0] + np.shape(Y)[0])
    return [sum_x - sum_y, mean_x, mean_y, mean_glob]

In [18]:
X1 = ['math', 'algebra', 'geometry', 'calculus', 'equation', 'computation', 'numbers', 'addition']
Y1 = ['poetry', 'art', 'dance', 'literature', 'novel', 'symphony', 'drama', 'sculpture']
Y1_vec = word_to_vector(Y1, model)
result1 = test_weat(X1, Y1, A1, B1, model)[0]
print(result1)


0.2569912484834471


**Etape 2** : vérifier ses caractéristiques statistiques

D'abord, on implémente le calcul de la taille d'effet (effect size). 

In [84]:
def effect_size(X,Y,A,B,model):
    result, mean_x, mean_y, mean_glob = test_weat(X,Y,A,B,model)
    ecart_type = 0
    X = word_to_vector(X, model)
    Y = word_to_vector(Y, model)
    A = word_to_vector(A, model)
    B = word_to_vector(B, model)
    for i in range(np.shape(X)[0]) :
        xi_AB = s_w_A_B(X[i], A, B, model)
        ecart_type += (xi_AB - mean_glob)**2
    for i in range(np.shape(Y)[0]) :
        yi_AB = s_w_A_B(Y[i], A, B, model)
        ecart_type += (yi_AB - mean_glob)**2
    ecart_type = ecart_type/(np.shape(X)[0] + np.shape(Y)[0]-1)
    ecart_type = np.sqrt(ecart_type)
    effect_size = (mean_x - mean_y)/ecart_type
    return effect_size

In [85]:
effect_size(X1, Y1, A1, B1, model)

0.988407824670586

Dans l'article, ils obtiennent un effect_size de 1,06 sur le word embedding GloVe. Les ordres de grandeur semblent cohérents. 

Maintenant, il faut implémenter le calcul de la p-valeur. 

Ce qui est décrit dans l'article : Let $ \{(X_i, Y_i)\}_i $ denote all the partitions of $ X \cup Y $ into two sets of equal size. The one-sided p-value of the permutation test is:

$ P_r \left[ s(X_i, Y_i, A, B) > s(X, Y, A, B) \right] $


La procédure décrite dans l'article indique qu'une permutation test doit être utilisée pour évaluer la p-valeur. Un test de permutation implique de réarranger aléatoirement les éléments entre les groupes (ici, X et Y) et de recalculer la statistique de test pour chaque arrangement. Cela permet de construire une distribution nulle de la statistique de test sous l'hypothèse nulle (pas d'association entre les groupes).

La p-valeur est ensuite calculée comme la proportion des arrangements aléatoires qui génèrent une statistique de test aussi extrême ou plus extrême que la statistique observée dans les données réelles.

In [92]:
import itertools

In [107]:
# Combine les ensembles X et Y
ensemble_combine = X1 + Y1
len(ensemble_combine)


16

In [114]:
 # Calcule la taille d'un seul ensemble après permutation
taille_ensemble = len(ensemble_combine) // 2

# Génère toutes les partitions possibles de X U Y en deux ensembles de taille égale
partitions = list(itertools.combinations(ensemble_combine, taille_ensemble))

print(type(partitions[0][0]))
print(len(partitions[0]))

<class 'str'>
8


In [118]:
def p_valeur(X,Y,A,B,model, stat_obs):
    # Combine les ensembles X et Y qui sont des listes de string
    ensemble_combine = X + Y
    # Calcule la taille d'un seul ensemble après permutation
    taille_ensemble = len(ensemble_combine) // 2
    # Génère toutes les partitions possibles de X U Y en deux ensembles de taille égale 
    #--> liste de tuples, chaque tuple contient 8 strings
    partitions = list(itertools.combinations(ensemble_combine, taille_ensemble))
    # Pour chaque partition, calcule la statistique de test et compte les arrangements extrêmes
    count = 0
    for partition in partitions:
        X_perm = list(partition)
        Y_perm = list(set(ensemble_combine) - set(X_perm))
        stat = test_weat(X_perm,Y_perm,A,B,model)[0]  #Calcule la statistique de test pour cette permutation
        if stat >= stat_obs:  # Compare la statistique de test de la permutation à la statistique observée
            count += 1
    # Calcule la p-valeur en divisant le nombre d'arrangements extrêmes par le nombre total de permutations
    p_valeur = count/ len(partitions)
    return p_valeur

In [119]:
p_valeur(X1, Y1, A1, B1, model, result1)

0.019502719502719503

L'article obtient une p-valeur de $10^-2$ sur le word embedding GloVe. 